## Installing / Importing Packages

The following packages are required for the notebook to work.

In [ ]:
# !git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
# !pip install keras==2.15.0
# !pip install gast==0.2.2
# !pip install protobuf==3.17.3
# !pip install tensorboard==2.10.0
# !pip install tensorflow-estimator==2.10.0

In [ ]:
# !pip install -r ./keras-retinanet/requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from keras_retinanet.utils.visualization import draw_box, draw_caption , label_color
from keras_retinanet.utils.image import preprocess_image, resize_image
from os.path import isfile, join
import matplotlib.pyplot as plt
import urllib
from os import listdir
import glob
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
import time
from keras_retinanet import models
from PIL import Image, ImageDraw, ImageFont
import csv
import tensorflow as tf

In [ ]:
#Checking the version of tensorflow
tf.__version__

#Checking if GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU is available")
    for gpu in gpus:
        print("Device name:", gpu.name)
else:
    print("No GPU available, using CPU instead")

In [ ]:
#Setting the path for the annotations
annPath="../Object_Detection_Dataset/CSV/Train/labels.csv"

In [ ]:
#Creating a folder to save the snapshots
if not os.path.exists('snapshots'):
  os.mkdir('snapshots')

## Retrieving the Unique Labels from CSV Function

This function takes two input parameters:

- **csvPath** - The location of the filename to retrieve information from.
- **index** - The column of the file to search through.

The file is first opened and the unique values from the column indicated by the index are retrieved and returned as a set.

In [ ]:
#Defining a function to get the unique values from a CSV file's column
def get_unique_values_from_csv(csvPath, index):
    unique_values = set()
    #Opening the CSV file
    with open(csvPath, 'r') as file:
        reader = csv.reader(file)
        #Iterating through the rows of the CSV file
        for row in reader:
            #Checking if the row has the required index
            if len(row) > index:
                unique_values.add(row[index])
    return unique_values

#Getting the unique labels from the CSV file
column_index = 5 
unique_values = get_unique_values_from_csv(annPath, column_index)
print("Unique Values:", unique_values)

## Inserting Information to CSV File Column

This function takes three input parameters:

- **csvFile** - The location of the filename to retrieve information from.
- **directoryName** - The information to be inserted. 
- **columnIndex** - The column of the file to insert through.

The file is first opened and all values are rewritten with the directoryName parameter concatenated to them. The file is then saved.

In [ ]:
#Defining a function to append the directory name to the column
def appendDirectoryToColumn(csvFile, directoryName, columnIndex):
    #Reading the CSV file and storing its contents
    with open(csvFile, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    #Appending the directory name to each value in the specified column
    for row in rows:
        row[columnIndex] = directoryName + row[columnIndex]

    #Writing the modified data back to the CSV file
    with open(csvFile, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

#Appending the directory name to the column
appendDirectoryToColumn(annPath, "images/", 0)

In [ ]:
#Creating a file to save the labels
with open('./Classes.csv', 'w') as file:
  for i, class_name in enumerate(unique_values):
    file.write(f'{class_name},{i}\n') 

## Installing and Loading Model

The model is first downloaded and has a few of its files altered. The reason behind these file changes was for evaluation purposes later on in the document.

In [ ]:
#Downloading the pre-trained model
url = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
model = './snapshots/resnet50_csv_v1.h5'
urllib.request.urlretrieve(url, model)

## Function to Replace Files

This function takes two parameters:

- **file_a** - The file to replace it with.
- **file_b** - The file to be replaced.

The file is overwritten by the other file's contents. This is used for evaluation purposes later on in the notebook.

In [ ]:
#Defining the function to replace the contents of a file
def copy_and_replace(file_a, file_b):
    try:
        #Reading the contents of the first file
        with open(file_a, 'r') as file_a_contents:
            data = file_a_contents.read()
        #Writing the contents of the first file to the second file
        with open(file_b, 'w') as file_b_contents:
            file_b_contents.write(data)

        print(f"Contents of {file_a} copied and replaced in {file_b} successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
#Replacing the files in the keras-retinanet library
evaluation_path = "./filesToReplace/modified_evaluate.py"
evaluation_copyTo_path = "./keras-retinanet/keras_retinanet/bin/evaluate.py"

eval_path = "./filesToReplace/modified_eval.py"
eval_copyTo_path = "./keras-retinanet/keras_retinanet/utils/eval.py"

visualise_path = "./filesToReplace/modified_visualization.py"
visualise_copyTo_path= "./keras-retinanet/keras_retinanet/utils/visualization.py"

copy_and_replace(evaluation_path, evaluation_copyTo_path)
copy_and_replace(eval_path, eval_copyTo_path)
copy_and_replace(visualise_path, visualise_copyTo_path)

## Training Model

The model is trained for 10 epochs, following that the model is saved and loaded.

In [ ]:
#Training the model
!python keras-retinanet/keras_retinanet/bin/train.py \
        --freeze-backbone --random-transform --gpu 0 --weights ./snapshots/resnet50_csv_v1.h5 \
         --batch-size 4 --steps 250 --epochs 10 --tensorboard-dir tensorboard csv ../Object_Detection_Dataset/CSV/Train/labels.csv ./Classes.csv

In [ ]:
#Selecting the latest model
print(glob.glob('./snapshots/*.h5'))
model_path = glob.glob('./snapshots/*.h5')[-2]

#Loading the model
print(model_path)
model = models.load_model(model_path, backbone_name='resnet50')

## Human Evaluation

The model is tested on a few unseen examples which are displayed to the user for human verification of the performance of the model.

In [ ]:
#Loading the model and converting it
model = models.load_model(glob.glob('./snapshots/*.h5')[-2], backbone_name='resnet50')
model = models.convert_model(model)

#Loading the classes
labels_to_names = pd.read_csv("./Classes.csv",header=None).T.loc[0].to_dict()

In [ ]:
#Appending the directory name to the column of the test CSV file
annPath="../Object_Detection_Dataset/CSV/Test/labels.csv"
appendDirectoryToColumn(annPath, "images/", 0)

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

#Selecting five images from the test folder
test_path = "../Object_Detection_Dataset/CSV/Test/images/"

images = [os.path.join(test_path,img) for img in os.listdir(test_path) if '.jpeg' in img]

images = images[:5]

## Function to Perform Inference on an Image

This function takes two parameters:

- **img_path** - The image file to perform inference on.
- **threshold** - The confidence threshold required.

This function opens the image and performs some transformations before the inference process can be done. A timer is also set up to check the processing time required for predictions to occur. Once the predictions are made, those below the confidence threshold are removed and the remaining ones have the bounding box, label and confidence value inset into the image.

In [ ]:
#Defining a function to perform inference on an image
def img_inference(img_path, threshold):
    #Opening the image
    image = Image.open(img_path)

    #Creating a draw object
    draw = ImageDraw.Draw(image)

    #Specifying font and size for the label
    font = ImageFont.load_default()

    #Preprocessing the image for the network
    image_np = read_image_bgr(img_path)
    image_np = preprocess_image(image_np)
    image_np, scale = resize_image(image_np)

    #Performing inference on the image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image_np, axis=0))
    print("Processing Time: ", time.time() - start)

    #Correcting the boxes for the image scale
    boxes /= scale

    #Iterating through the boxes
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        #Checking if the score is below the threshold
        if score < threshold:
            break

        #Extracting the box coordinates
        x1, y1, x2, y2 = box.astype(int)

        #Drawing the rectangle around the object
        draw.rectangle([x1, y1, x2, y2], outline="green", width=2)

        #Drawing the label and score
        label_text = "{} {:.2f}".format(labels_to_names[label], score)
        draw.text((x1, y1 - 15), label_text, fill="green", font=font)
        print(label_text)
    
    #Displaying the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

#Performing inference on the images
for img in images:
    img_inference(img, 0.5)



## Testing the Model

The model is converted to an inference model and is tested on the Testing Set. Its results are depicted through the use of a mAP graph for each label as well as a precision recall graph.

In [ ]:
#Convert training model to inference model

#NOTE: CHANGE THE PATHS TO THE CORRECT PATHS

!python keras-retinanet/keras_retinanet/bin/convert_model.py ./snapshots/resnet50_csv_10.h5 ./snapshots/inference_resnet50_csv.h5

!python keras-retinanet/keras_retinanet/bin/evaluate.py --gpu 0 --score-threshold 0.0 --save-path ./evaluation --no-resize\
        csv ../Object_Detection_Dataset/CSV/Test/labels.csv ./Classes.csv ./snapshots/inference_resnet50_csv.h5

In [ ]:
#Loading the evaluation results
evaluation_results = np.load('./evaluation_results.npy', allow_pickle=True)

average_precisions = evaluation_results[0]
inference_time     = evaluation_results[1]
precision_recall   = evaluation_results[2]

## Calculating Mean Average Precision Value

The mean average precision value is calculated alongside the average precision values for each label.

In [ ]:
#Reading the classes from the CSV file
df = pd.read_csv('./Classes.csv', header=None)
classes = df[0].tolist()

#Retrieving the average precisions for each class
aps = [average_precision[0] for label, average_precision in average_precisions.items()]

#Displaying the average precisions for each class
print('\033[35m' + 'Average Precisions for each class:' + '\033[0m')
for idx, ap in enumerate(aps):
    print(f'{classes[idx]:15}: {ap}')

#Displaying the mean average precision
mean_average_precision = np.mean(aps)
print(f'\n\033[37m' + 'Mean Average Precision (mAP):' + '\033[0m')
print(f'{mean_average_precision}')

## Displaying Average Precision Graph

A bar chart is displayed depicting the Average Precision Value of each label.

In [ ]:
#Plotting the average precisions for each class
plt.figure(figsize=(20, 10))
plt.bar(list(average_precisions.keys()), list(aps), color='blue')

#Putting text on the bar chart
for idx, val in enumerate(aps):
    plt.text(idx, val, round(val, 3), horizontalalignment='center', verticalalignment='bottom', fontdict={'fontweight': 500, 'size': 12})

#Setting the x-ticks to correspond to all labels
plt.gca().set_xticks(range(len(classes)))
plt.xticks(rotation=45, fontsize=12)

#Setting the labels to the actual class labels
plt.gca().set_xticklabels(classes)
plt.xlabel("Labels", fontsize=14)
plt.ylabel("Mean Average Precision", fontsize=14)
plt.title("Mean Average Precision vs Labels (Keras RetinaNet)", fontsize=20)
plt.tight_layout()
plt.show()

## Displaying Precision-Recall Curve

The precision-recall curve is displayed using the results calculated earlier.

In [ ]:
#Plotting the precision-recall curve
plt.figure(figsize=(20, 10))
#Iterating through the precision-recall values
for label, (precision_list, recall_list) in precision_recall.items():
    plt.plot(recall_list, precision_list, label=str(classes[label]) + " (AP: {:.4f})".format(aps[label]))
plt.xlabel("Recall", fontsize=14)
plt.ylabel("Precision", fontsize=14)
plt.title("Precision vs Recall Curve (Keras RetinaNet)", fontsize=20)
plt.legend(fontsize=12)
plt.grid()
plt.tight_layout()
plt.show()


## Resetting Files Function

This function takes two input parameters:

- **csvFile** - The location of the filename to retrieve information from.
- **columnIndex** - The column of the file to search through.

The file is first opened and the directory of the images is removed.

In [ ]:
annPathTrain="../Object_Detection_Dataset/CSV/Train/labels.csv"
annPathTest="../Object_Detection_Dataset/CSV/Test/labels.csv"

#Defining a function to remove the directory name from a CSV file's column
def removeDirectoryFromColumn(csvFile, columnIndex):
    #Reading the CSV file and storing its contents
    with open(csvFile, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    #Removing the directory name from each value in the specified column
    for row in rows:
        row[columnIndex] = os.path.basename(row[columnIndex])

    #Writing the modified data back to the CSV file
    with open(csvFile, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

removeDirectoryFromColumn(annPathTrain, 0)
removeDirectoryFromColumn(annPathTest, 0)